In [2]:
import bpy
import numpy as np
from IPython.display import display, Image
import os
import mathutils
from tqdm import tqdm
from typing import List, Dict
import bmesh

import matplotlib.pyplot as plt

print("zz")

zz


In [3]:
FOLDER = "C:\\Users\\pierr\\Documents\\Blender\\GeometryProcessing_Python\\"
FOLDER = "C:\\Users\\Pierre.Gilibert\\OneDrive - ARVERNE\\Documents\\Divers\\blender"
bpy.ops.wm.open_mainfile(filepath=os.path.join(FOLDER, "blender_notebook_v00.blend"))

{'FINISHED'}

In [ ]:
def rotate_axis_angle(vec, axis, angle):
    c, s = np.cos(angle), np.sin(angle)
    # notation : p = vec, u = axis
    vec_dot_axis = np.dot(vec, axis) # u.p
    axis_x_vec = np.cross(axis, vec) # u x p

    return (1-c) * vec_dot_axis * axis + c * vec + s * axis_x_vec # (1-cos(θ))(u.p)u + cos(θ)p + sin(θ)(u x p)

def rotate_coordinate_system(u, v, nf):
    n = np.cross(u,v)
    n = n/np.linalg.norm(n)
    axis = np.cross(n,nf)
    axis = axis/np.linalg.norm(axis)
    angle = np.arccos(np.dot(n,nf))

    # n_r = rotate_axis_angle(n, axis, angle)
    u_r = rotate_axis_angle(u, axis, angle)
    v_r = rotate_axis_angle(v, axis, angle)
    return u_r, v_r

class MYMesh:
    def __init__(self):
        self.bm = bmesh.new()
        self.facecorners = []
        self.vert2fc = {}
        self.fc_attributes = {} # facecorners attributes
        self.vertex_attributes = {}
        self.face_attributes = {}
        self.fc_areas = {}
        self.face_vertex_to_facecorner = {}

    def from_mesh(self, mesh_data):
        """ Mimic the bmesh from_mesh function. """
        self.bm.from_mesh(mesh_data)
        self.vert2fc = {v:[] for v in self.verts}
        self.facecorners = []
        for face in self.faces:
            self.face_vertex_to_facecorner[face] = {}
            for loop in face.loops:
                self.vert2fc[loop.vert].append(loop)
                self.facecorners.append(loop)
                self.fc_areas[loop] = 0
                self.face_vertex_to_facecorner[face][loop.vert] = loop

    def to_mesh(self, mesh_data):
        """ Mimic the bmesh to_mesh function. """
        self.bm.to_mesh(mesh_data)
    
    # Here you can add any other bmesh methods you need to use:
    def free(self):
        """ Mimic the bmesh free function. """
        self.bm.free()

    # Add a custom method to directly access the internal bmesh:
    def __getattr__(self, item):
        # Delegate attribute access to the internal bmesh instance
        return getattr(self.bm, item)
    
    def ensure_lookup_tables(self):
        self.verts.ensure_lookup_table()
        self.edges.ensure_lookup_table()
        self.faces.ensure_lookup_table()

    def calculate_internal_angles(self):
        self.fc_attributes["internal_angle"] = [fc.calc_angle() for fc in self.facecorners]

    def _retrieve_fc_given_fv(self, face, vertex):
            """
            Return the facecorner (loop) of face and vertex, None if not found
            """
            # for fc in self.vert2fc[vertex]:
            #         if fc.face == face:
            #             return fc
            # return None
            return self.face_vertex_to_facecorner[face][vertex]

    def _retrieve_internal_angles_and_other_vertices(self, f, vi):
        vj, vk = [other for other in f.verts if other!=vi]
            
        fci = self._retrieve_fc_given_fv(f, vi)
        fcj = self._retrieve_fc_given_fv(f, vj)
        fck = self._retrieve_fc_given_fv(f, vk)

        if fci is None or fcj is None or fck is None:
            raise ValueError(f"Unable to find a facecorner (loop) associated to vertex {vi.index}, {vj.index} or {vk.index} at face {f.index}")
        alpha_i = fci.calc_angle()
        alpha_j = fcj.calc_angle()
        alpha_k = fck.calc_angle()
        return vj, vk, alpha_i, alpha_j, alpha_k

    def _calculate_vertex_area(self, vi):
        """
        Calculate the mixed area at vertex vi (and its face corners)
        """
        vi_area = 0
        for f in vi.link_faces:

            vj, vk, alpha_i, alpha_j, alpha_k = self._retrieve_internal_angles_and_other_vertices(f, vi)

            if alpha_i < np.pi/2 and alpha_j < np.pi/2 and alpha_k < np.pi/2: # Non obtuse triangle
                eij = vj.co - vi.co
                eik = vk.co - vi.co
                local_area = (1/8) * (eij.dot(eij) / np.tan(alpha_k) + eik.dot(eik) / np.tan(alpha_j))
            elif alpha_i > np.pi/2:
                local_area = f.calc_area()/2
            else:
                local_area = f.calc_area()/4

            fci = self._retrieve_fc_given_fv(f, vi)
            if self.fc_areas[fci] != 0:
                raise ValueError("wtf")
            self.fc_areas[fci] = local_area
            vi_area += local_area


        return vi_area
    
    def calculate_vertices_area(self):
        self.vertex_attributes["vertex_area"] = [self._calculate_vertex_area(v) for v in self.verts]

    def _calculate_mean_curvature_normal_operator_vertex(self, vi):
        mean_crv_i = mathutils.Vector([0.0, 0.0, 0.0])
        for f in vi.link_faces:
            vj, vk, _, alpha_j, alpha_k = self._retrieve_internal_angles_and_other_vertices(f, vi)
            eij = vj.co - vi.co
            eik = vk.co - vi.co
            mean_crv_i = mean_crv_i + (eij/np.tan(alpha_k) + eik/np.tan(alpha_j))

        K_i = mean_crv_i/(2*self.vertex_attributes["vertex_area"][vi.index])
        vi_normal = K_i/np.linalg.norm(K_i) # vertex normal
        ejk = vk.co - vj.co
        # ui = np.array([np.linalg.norm(ejk), np.linalg.norm(eik), np.linalg.norm(eij)])
        ui = np.array([np.linalg.norm(eij), np.linalg.norm(ejk), np.linalg.norm(ejk)])
        ui = np.cross(ui, vi_normal)
        ui = ui/np.linalg.norm(ui)
        vi = np.cross(vi_normal, ui)


        return K_i, vi_normal, ui, vi

    def calculate_mean_curvature_vector(self):
        """
        return the mean curvature normal operator (vector in R3) at each vertex of the mesh
        """
        if not "vertex_area" in self.vertex_attributes:
            self.calculate_vertices_area()

        self.vertex_attributes["mean_curvature_operator"] = []
        self.vertex_attributes["vertex_normal"] = []
        self.vertex_attributes["u"] = []
        self.vertex_attributes["v"] = []

        for v in self.verts:
            K, v_normal, ui, vi = self._calculate_mean_curvature_normal_operator_vertex(v)
            self.vertex_attributes["mean_curvature_operator"].append(K)
            self.vertex_attributes["vertex_normal"].append(v.normal)
            # self.vertex_attributes["vertex_normal"].append(v_normal)
            self.vertex_attributes["u"].append(ui)
            self.vertex_attributes["v"].append(vi)
        # self.vertex_attributes["mean_curvature_operator"] = [self._calculate_mean_curvature_normal_operator_vertex(v) for v in self.verts]
            
    def calc_curvatures(self):
        self.face_attributes["2nd_fundamental_tensor_f"] = []
        self.vertex_attributes["2nd_fundamental_tensor_v"] = [np.zeros(3) for _ in range(len(self.verts))]
        for f in self.faces:
            # vertices and edges of the face
            vi, vj, vk = f.verts
            eij = vj.co - vi.co
            ejk = vk.co - vj.co
            eki = vi.co - vk.co

            # Coordinate system of the facd
            nf = f.normal
            uf = eij/np.linalg.norm(eij) 
            vf = np.cross(nf, uf)
            vf = vf / np.linalg.norm(vf)

            # vertex normals at the vertices            
            ni = self.vertex_attributes["vertex_normal"][vi.index]
            nj = self.vertex_attributes["vertex_normal"][vj.index]
            nk = self.vertex_attributes["vertex_normal"][vk.index]

            eij_uf, eij_vf = np.dot(eij, uf), np.dot(eij, vf)
            ejk_uf, ejk_vf = np.dot(ejk, uf), np.dot(ejk, vf)
            eki_uf, eki_vf = np.dot(eki, uf), np.dot(eki, vf)
            A = np.array([
                [eij_uf, eij_vf, 0],
                [0, eij_uf, eij_vf],
                [ejk_uf, ejk_vf, 0],
                [0, ejk_uf, ejk_vf],
                [eki_uf, eki_vf, 0],
                [0, eki_uf, eki_vf]
            ])
            b = np.array([
                np.dot(nj-ni, uf),
                np.dot(nj-ni, vf),
                np.dot(nk-nj, uf),
                np.dot(nk-nj, vf),
                np.dot(ni-nk, uf),
                np.dot(ni-nk, vf)
            ])
            # solve Ax = b (least square)
            x, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
            
            # x = [x0, x1, x2] (vector in R3) containes the coordinate of the curvature tensor [[x0,x1],[x1,x2]]
            self.face_attributes["2nd_fundamental_tensor_f"].append(np.array(x))

            # retrieve the face_corners of each vertex:
            fci = self._retrieve_fc_given_fv(f, vi)
            fcj = self._retrieve_fc_given_fv(f, vj)
            fck = self._retrieve_fc_given_fv(f, vk)
            # compute the weight of the corner (area corner/area vertex)
            wci = self.fc_areas[fci] / self.vertex_attributes["vertex_area"][vi.index]
            wcj = self.fc_areas[fcj] / self.vertex_attributes["vertex_area"][vj.index]
            wck = self.fc_areas[fck] / self.vertex_attributes["vertex_area"][vk.index]

            u_i, v_i = self.vertex_attributes["u"][vi.index], self.vertex_attributes["v"][vi.index]
            u_j, v_j = self.vertex_attributes["u"][vj.index], self.vertex_attributes["v"][vj.index]
            u_k, v_k = self.vertex_attributes["u"][vk.index], self.vertex_attributes["v"][vk.index]
            
            kui, kuvi, kvi = self.project_curvature_tensor(uf, vf, nf, x[0], x[1], x[2], u_i, v_i)
            self.vertex_attributes["2nd_fundamental_tensor_v"][vi.index] = self.vertex_attributes["2nd_fundamental_tensor_v"][vi.index] + wci * np.array([kui, kuvi, kvi])
            kuj, kuvj, kvj = self.project_curvature_tensor(uf, vf, nf, x[0], x[1], x[2], u_j, v_j)
            self.vertex_attributes["2nd_fundamental_tensor_v"][vj.index] = self.vertex_attributes["2nd_fundamental_tensor_v"][vj.index] + wcj * np.array([kuj, kuvj, kvj])
            kuk, kuvk, kvk = self.project_curvature_tensor(uf, vf, nf, x[0], x[1], x[2], u_k, v_k)
            self.vertex_attributes["2nd_fundamental_tensor_v"][vk.index] = self.vertex_attributes["2nd_fundamental_tensor_v"][vk.index] + wck * np.array([kuk, kuvk, kvk])


    def calc_principal_curvatures(self):
        self.vertex_attributes["my_k1"] = []
        self.vertex_attributes["my_k2"] = []
        self.vertex_attributes["my_e1"] = []
        self.vertex_attributes["my_e2"] = []
        for v in self.verts:
            u_v, v_v = self.vertex_attributes["u"][v.index], self.vertex_attributes["v"][v.index]
            ku, kuv, kv = self.vertex_attributes["2nd_fundamental_tensor_v"][v.index]
            c, s, tt = 1,0,0 # cos, sin, tangent
            if kuv != 0:
                # Jacobi rotation to diagonalize the second fundamental matrix
                h = 0.5 * (kv - ku) / kuv
                if h < 0:
                    tt = 1 / (h - np.sqrt(1 + h * h))
                else:
                    tt = 1 / (h + np.sqrt(1 + h * h))

                # Calculate cosine and sine for the rotation
                c = 1 / np.sqrt(1 + tt * tt)
                s = tt * c
            # Compute the principal curvatures
            k1 = ku - tt * kuv
            k2 = kv + tt * kuv

            if abs(k1) >= abs(k2):
                e1 = c * u_v - s * v_v
            else: # swap k1 and k2:
                k1, k2 = k2, k1
                e1 = s * u_v + c * v_v
            n_v = self.vertex_attributes["vertex_normal"][v.index]
            e2 = np.cross(n_v, e1)

            self.vertex_attributes["my_k1"].append(k1)
            self.vertex_attributes["my_k2"].append(k2)
            self.vertex_attributes["my_e1"].append(e1)
            self.vertex_attributes["my_e2"].append(e2)


        
    def project_curvature_tensor(self, uf, vf, nf, old_ku, old_kuv, old_kv, uv, vv):
        
        ur, vr = rotate_coordinate_system(uv, vv, nf)
        u_proj = [np.dot(ur, uf), np.dot(ur, vf)]
        v_proj = [np.dot(vr, uf), np.dot(vr, vf)]
        II = np.array([
            [old_ku, old_kuv],
            [old_kuv, old_kv]
        ])
        ku = np.dot(u_proj, np.dot(II, u_proj))
        kuv = np.dot(u_proj, np.dot(II, v_proj))
        kv = np.dot(v_proj, np.dot(II, v_proj))
        return ku, kuv, kv

    def _calculate_angle_defect(self, v):
        """
        Calculate the angle defect at the specified vertex
        """
        angle_sum = 0
        for fc in self.vert2fc[v]:
            angle_sum += fc.calc_angle()

        return (2*np.pi - angle_sum)/self.vertex_attributes["vertex_area"][v.index]
    
    def calculate_gaussian_curvature(self):
        """
        return the gaussian curvature (angle defect) at each vertex of the mesh
        """
        if not "vertex_area" in self.vertex_attributes:
            self.calculate_vertices_area()
        self.vertex_attributes["gaussian_curvature"] = [self._calculate_angle_defect(v) for v in self.verts]

    def calculate_kijN(self):
        if not "vertex_area" in self.vertex_attributes:
            self.calculate_vertices_area()
        if not "gaussian_curvature" in self.vertex_attributes:
            self.calculate_gaussian_curvature()
        if not "mean_curvature_operator" in self.vertex_attributes:
            self.calculate_mean_curvature_vector()
        kg = np.array(self.vertex_attributes["gaussian_curvature"])
        K = np.array(self.vertex_attributes["mean_curvature_operator"])
        kh = 0.5 * np.linalg.norm(K, axis=1)
        delta = np.maximum(0, kh*kh - kg)
        print(kg)
        print(kh)
            
obj = bpy.data.objects["Suzanne"]
bpy.ops.object.mode_set(mode='OBJECT')
mesh = obj.data

bm = MYMesh()
bm.from_mesh(mesh)
# bm.from_mesh(mesh)
bm.ensure_lookup_tables()
bm.calculate_internal_angles()
bm.calculate_vertices_area()
bm.calculate_mean_curvature_vector()
bm.calculate_gaussian_curvature()
bm.calc_curvatures()
bm.calc_principal_curvatures()

attribute_names = ["vertex_area", "internal_angle", "mean_curvature_operator", "gaussian_curvature",
                    "mean_curvature", "e1", "e2", "VertexNormal", "vertex_normal",
                      "u", "v", "2nd_fundamental_tensor", "my_k1", "my_k2", "my_e1", "my_e2"]
for name in attribute_names:
    for attr in mesh.attributes:
        if attr.name.startswith(name):
            mesh.attributes.remove(attr)



kH, kG = np.array(bm.vertex_attributes["mean_curvature_operator"]), np.array(bm.vertex_attributes["gaussian_curvature"])
kH = 0.5 * np.linalg.norm(kH, axis=1)
delta = kH*kH - kG
# delta = np.maximum(delta, 0)
# k1 = kH + np.sqrt(delta)
# k2 = kH - np.sqrt(delta)
# k1,k2


attr = mesh.attributes.new(name="vertex_area", type='FLOAT', domain='POINT')
attr.data.foreach_set('value', bm.vertex_attributes["vertex_area"])

attr = mesh.attributes.new(name="internal_angle", type='FLOAT', domain='CORNER')
attr.data.foreach_set('value', bm.fc_attributes["internal_angle"])

attr = mesh.attributes.new(name="mean_curvature_operator", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector',np.array(bm.vertex_attributes["mean_curvature_operator"]).flatten())

attr = mesh.attributes.new(name="mean_curvature", type='FLOAT', domain='POINT')
attr.data.foreach_set('value', kH)

attr = mesh.attributes.new(name="mean_curvature_normal", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector',np.array([x/np.linalg.norm(x) for x in bm.vertex_attributes["mean_curvature_operator"]]).flatten())


attr = mesh.attributes.new(name="gaussian_curvature", type='FLOAT', domain='POINT')
attr.data.foreach_set('value', kG)

attr = mesh.attributes.new(name="vertex_normal", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector', np.array(bm.vertex_attributes["vertex_normal"]).flatten())

attr = mesh.attributes.new(name="u", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector', np.array(bm.vertex_attributes["u"]).flatten())


attr = mesh.attributes.new(name="v", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector', np.array(bm.vertex_attributes["v"]).flatten())

attr = mesh.attributes.new(name="2nd_fundamental_tensor_v", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector', np.array(bm.vertex_attributes["2nd_fundamental_tensor_v"]).flatten())

attr = mesh.attributes.new(name="2nd_fundamental_tensor_f", type='FLOAT_VECTOR', domain='FACE')
attr.data.foreach_set('vector', np.array(bm.face_attributes["2nd_fundamental_tensor_f"]).flatten())

for name in ["my_k1", "my_k2"]:
    attr = mesh.attributes.new(name=name, type='FLOAT', domain='POINT')
    attr.data.foreach_set('value', bm.vertex_attributes[name])

for name in ["my_e1", "my_e2"]:
    attr = mesh.attributes.new(name=name, type='FLOAT_VECTOR', domain='POINT')
    attr.data.foreach_set('vector', np.array(bm.vertex_attributes[name]).flatten())


kg_sum = np.sum(kG)
kh_sum = np.sum(kH)
va_sum = np.sum(bm.vertex_attributes["vertex_area"])
delta_sum = np.sum(delta)
old_kgsum, old_khsum, old_va_sum = 6221835.615298654, 109963.44637969346, 11.095911415050558
old_delta_sum = -951188.585732151

print(kg_sum, kh_sum, va_sum, delta_sum)
print(kg_sum/old_kgsum, kh_sum/old_khsum, va_sum/old_va_sum, delta_sum/old_delta_sum)

6221835.615298654 109963.44637969346 11.095911415050558 -951188.585732151
1.0 1.0 1.0 1.0


In [4]:
nn = np.cross(bm.vertex_attributes["u"], bm.vertex_attributes["v"])
nn[:5]


array([[ 0.66420704, -0.57190466, -0.4814083 ],
       [-0.66420597, -0.57190734, -0.4814066 ],
       [-0.60088038,  0.59967887,  0.52851486],
       [ 0.60088128,  0.59967744,  0.52851552],
       [-0.21065105, -0.89335108,  0.39692578]])

In [5]:
bm.vertex_attributes["vertex_normal"][:5]

[Vector((0.5833245515823364, -0.6179108023643494, -0.5271798968315125)),
 Vector((-0.5833247900009155, -0.6179096102714539, -0.527181088924408)),
 Vector((0.5998024940490723, -0.5981992483139038, -0.5314081311225891)),
 Vector((-0.5998022556304932, -0.5981989502906799, -0.5314087271690369)),
 Vector((0.702803373336792, -0.38308364152908325, -0.5994283556938171))]

In [6]:
for i in range(5):
    up, vp = bm.vertex_attributes["u"][i], bm.vertex_attributes["v"][i]
    n_vector = np.cross(up,vp)
    r_old_u, r_old_v = rotate_coordinate_system(up, vp, n_vector)
    print(n_vector, nn[i])


[ 0.66420704 -0.57190466 -0.4814083 ] [ 0.66420704 -0.57190466 -0.4814083 ]
[-0.66420597 -0.57190734 -0.4814066 ] [-0.66420597 -0.57190734 -0.4814066 ]
[-0.60088038  0.59967887  0.52851486] [-0.60088038  0.59967887  0.52851486]
[0.60088128 0.59967744 0.52851552] [0.60088128 0.59967744 0.52851552]
[-0.21065105 -0.89335108  0.39692578] [-0.21065105 -0.89335108  0.39692578]


C:\Users\pierr\AppData\Local\Temp\ipykernel_10600\544956718.py:13: RuntimeWarning: invalid value encountered in divide
  axis = axis/np.linalg.norm(axis)
C:\Users\pierr\AppData\Local\Temp\ipykernel_10600\544956718.py:14: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(np.dot(n,nf))


In [7]:
Npoints = 2
u = np.random.rand(3*Npoints)*2-1
v = np.random.rand(3*Npoints)*2-1
nf = np.random.rand(3*Npoints)*2-1
u,v,nf = u.reshape((-1,3)),v.reshape((-1,3)),nf.reshape((-1,3))

u = u/np.linalg.norm(u, axis=1)[:,None]
v = v/np.linalg.norm(v, axis=1)[:,None]
nf = nf/np.linalg.norm(nf, axis=1)[:,None]

nf[:,0] = 0
nf[:,1] = 0
nf[:,2] = 1
u[0] = [0.38, 0.725, -0.574]
v[0] = [-0.596, 0.273, -0.755]

In [8]:

def rotate_axis_angle(vec, axis, angle):
    c, s = np.cos(angle), np.sin(angle)
    # notation : p = vec, u = axis
    vec_dot_axis = np.sum(vec*axis, axis=1)[:,np.newaxis] # u.p
    axis_x_vec = np.cross(axis, vec) # u x p

    return (1-c) * vec_dot_axis * axis + c * vec + s * axis_x_vec # (1-cos(θ))(u.p)u + cos(θ)p + sin(θ)(u x p)

def rotate_coord_system_axisangle(u,v,nf):
    n = np.cross(u,v)
    n = n/np.linalg.norm(n, axis=1)[:,np.newaxis]
    axis = np.cross(n,nf)
    axis = axis/np.linalg.norm(axis, axis=1)[:,np.newaxis]
    angle = np.arccos(np.sum(n*nf, axis=1))[:,np.newaxis]

    # n_r = rotate_axis_angle(n, axis, angle)
    u_r = rotate_axis_angle(u, axis, angle)
    v_r = rotate_axis_angle(v, axis, angle)
    return u_r, v_r

def rotate_coord_system(u,v,nf):
    """
    rotate the vectors u and v to the plane of normal nf
    Args:
        u (np.array (N,3)) : 1st set of vector to be rotated
        v (np.array (N,3)) : 2nd set of vector to be rotated
        nf (np.array (N,3)) : normal vectors of the plane to rotate into
    Output:
        new_u, new_v (np.array(N, 3)) : rotated u and v (orthogonal to nf)
    """
    n = np.cross(u,v)
    n = n/np.linalg.norm(n, axis=1)[:,None]

    dot = np.sum(nf*n, axis=1)[:,None] # np.dot(nf,n) : nf.n
    perp = nf - dot*n
    dperp = (nf + n)/(1+dot)
    new_u = u - dperp * np.sum(perp*u, axis=1)[:,None] # u - dperp * (perp.u)
    new_v = v - dperp * np.sum(perp*v, axis=1)[:,None]

    threshold = np.where(dot<=-1)[0]
    new_u[threshold] = -u[threshold]
    new_v[threshold] = -v[threshold]

    return new_u, new_v




new_u, new_v = rotate_coord_system(u, v, nf)
print(new_u)
print(new_v)


[[ 2.25330307e-01  9.74026310e-01 -1.11022302e-16]
 [ 1.65393935e-01 -9.86227583e-01  0.00000000e+00]]
[[-7.99441844e-01  6.00552028e-01 -1.11022302e-16]
 [ 9.97808438e-01  6.61688824e-02  0.00000000e+00]]


In [9]:

def rotate_coord_system(u,v,nf):
    """
    rotate the vectors u and v to the plane of normal nf
    Args:
        u (np.array (N,3)) : 1st set of vector to be rotated
        v (np.array (N,3)) : 2nd set of vector to be rotated
        nf (np.array (N,3)) : normal vectors of the plane to rotate into
    Output:
        new_u, new_v (np.array(N, 3)) : rotated u and v (orthogonal to nf)
    """
    n = np.cross(u,v)
    n = n/np.linalg.norm(n, axis=1)[:,None]

    dot = np.sum(nf*n, axis=1)[:,None] # np.dot(nf,n) : nf.n
    perp = nf - dot*n
    dperp = (nf + n)/(1+dot)
    new_u = u - dperp * np.sum(perp*u, axis=1)[:,None] # u - dperp * (perp.u)
    new_v = v - dperp * np.sum(perp*v, axis=1)[:,None]

    threshold = np.where(dot<=-1)[0]
    new_u[threshold] = -u[threshold]
    new_v[threshold] = -v[threshold]

    return new_u, new_v


def get_face_normals(mesh:MYMesh):
    return np.array([f.normal for f in mesh.faces])


    

face_n = get_face_normals(bm)
# calc_vertex_normal(bm, face_n)


# new_u, new_v = rotate_coord_system(u,v,nf)
# print(np.sum(new_u*new_v, axis=1), np.sum(u*v, axis=1), np.sum(new_u*nf, axis=1), np.sum(new_v*nf, axis=1))

In [50]:
def calc_face_normals(FV):
    """
    Calculate the normal vectors for each face of a mesh.
    
    Parameters:
    FV (dict): A dictionary representing a triangle mesh in face-vertex structure.
               It should have keys 'vertices' (Nv x 3 matrix) and 'faces' (Nf x 3 matrix).
    
    Returns:
    FaceNormals (numpy.ndarray): (Nf x 3) matrix containing the normal vector at each face.
    """
    # Get all edge vectors
    e0 = FV['vertices'][FV['faces'][:, 2], :] - FV['vertices'][FV['faces'][:, 1], :]
    e1 = FV['vertices'][FV['faces'][:, 0], :] - FV['vertices'][FV['faces'][:, 2], :]

    # Calculate normal of each face using the cross product of two edges
    FaceNormals = np.cross(e0, e1)

    # Normalize the face normals
    FaceNormals /= np.linalg.norm(FaceNormals, axis=1)[:, np.newaxis]

    return FaceNormals

def calc_vertex_normals(FV, N):
    """
    Calculate vertex normals and voronoi areas for each vertex.
    
    Parameters:
    FV (dict): A dictionary representing a triangle mesh in face-vertex structure. 
               It should have keys 'vertices' (Nv x 3 matrix) and 'faces' (Nf x 3 matrix).
    N (numpy.ndarray): Face normals, an (Nf x 3) matrix.
    
    Returns:
    VertexNormals (numpy.ndarray): (Nv x 3) matrix of normals at each vertex.
    Avertex (numpy.ndarray): (Nv x 1) voronoi area at each vertex.
    Acorner (numpy.ndarray): (Nf x 3) slice of the voronoi area at each face corner.
    up (numpy.ndarray): (Nv x 3) part of the initial vertex coordinate system.
    vp (numpy.ndarray): (Nv x 3) part of the initial vertex coordinate system.
    """
    print('Calculating vertex normals... Please wait')

    # Get all edge vectors
    e0 = FV['vertices'][FV['faces'][:, 2], :] - FV['vertices'][FV['faces'][:, 1], :]
    e1 = FV['vertices'][FV['faces'][:, 0], :] - FV['vertices'][FV['faces'][:, 2], :]
    e2 = FV['vertices'][FV['faces'][:, 1], :] - FV['vertices'][FV['faces'][:, 0], :]
    # Normalize edge vectors
    e0_norm = e0 / np.linalg.norm(e0, axis=1)[:, np.newaxis]
    e1_norm = e1 / np.linalg.norm(e1, axis=1)[:, np.newaxis]
    e2_norm = e2 / np.linalg.norm(e2, axis=1)[:, np.newaxis]

    # Calculate edge lengths
    de0 = np.linalg.norm(e0, axis=1)
    de1 = np.linalg.norm(e1, axis=1)
    de2 = np.linalg.norm(e2, axis=1)
    l2 = np.vstack((de0**2, de1**2, de2**2)).T

    # Calculate weights for the voronoi area calculation
    ew = np.vstack((l2[:, 0] * (l2[:, 1] + l2[:, 2] - l2[:, 0]),
                    l2[:, 1] * (l2[:, 2] + l2[:, 0] - l2[:, 1]),
                    l2[:, 2] * (l2[:, 0] + l2[:, 1] - l2[:, 2]))).T

    # Heron's formula for triangle area
    s = (de0 + de1 + de2) / 2
    Af = np.sqrt(s * (s - de0) * (s - de1) * (s - de2))

    # Initialize outputs
    Acorner = np.zeros((FV['faces'].shape[0], 3))
    Avertex = np.zeros((FV['vertices'].shape[0],))
    VertexNormals = np.zeros((FV['vertices'].shape[0], 3))
    up = np.zeros((FV['vertices'].shape[0], 3))
    vp = np.zeros((FV['vertices'].shape[0], 3))

    # Calculate vertex normals and areas
    for i in range(FV['faces'].shape[0]):
        # Calculate weights according to N.Max [1999]
        wfv1 = Af[i] / (de1[i]**2 * de2[i]**2)
        wfv2 = Af[i] / (de0[i]**2 * de2[i]**2)
        wfv3 = Af[i] / (de1[i]**2 * de0[i]**2)

        # Accumulate weighted normals for each vertex
        VertexNormals[FV['faces'][i, 0], :] += wfv1 * N[i, :]
        VertexNormals[FV['faces'][i, 1], :] += wfv2 * N[i, :]
        VertexNormals[FV['faces'][i, 2], :] += wfv3 * N[i, :]

        # Calculate voronoi areas (Meyer et al. [2002])
        if ew[i, 0] <= 0:
            Acorner[i, 1] = -0.25 * l2[i, 2] * Af[i] / np.dot(e0[i, :], e2[i, :])
            Acorner[i, 2] = -0.25 * l2[i, 1] * Af[i] / np.dot(e0[i, :], e1[i, :])
            Acorner[i, 0] = Af[i] - Acorner[i, 1] - Acorner[i, 2]
        elif ew[i, 1] <= 0:
            Acorner[i, 2] = -0.25 * l2[i, 0] * Af[i] / np.dot(e1[i, :], e0[i, :])
            Acorner[i, 0] = -0.25 * l2[i, 2] * Af[i] / np.dot(e1[i, :], e2[i, :])
            Acorner[i, 1] = Af[i] - Acorner[i, 0] - Acorner[i, 2]
        elif ew[i, 2] <= 0:
            Acorner[i, 0] = -0.25 * l2[i, 1] * Af[i] / np.dot(e2[i, :], e1[i, :])
            Acorner[i, 1] = -0.25 * l2[i, 0] * Af[i] / np.dot(e2[i, :], e0[i, :])
            Acorner[i, 2] = Af[i] - Acorner[i, 0] - Acorner[i, 1]
        else:
            ewscale = 0.5 * Af[i] / (ew[i, 0] + ew[i, 1] + ew[i, 2])
            Acorner[i, 0] = ewscale * (ew[i, 1] + ew[i, 2])
            Acorner[i, 1] = ewscale * (ew[i, 0] + ew[i, 2])
            Acorner[i, 2] = ewscale * (ew[i, 0] + ew[i, 1])

        # Accumulate vertex areas
        Avertex[FV['faces'][i, 0]] += Acorner[i, 0]
        Avertex[FV['faces'][i, 1]] += Acorner[i, 1]
        Avertex[FV['faces'][i, 2]] += Acorner[i, 2]

        # Calculate initial coordinate system
        up[FV['faces'][i, 0], :] = e2_norm[i, :]
        up[FV['faces'][i, 1], :] = e0_norm[i, :]
        up[FV['faces'][i, 2], :] = e1_norm[i, :]

    # Normalize vertex normals
    VertexNormals /= np.linalg.norm(VertexNormals, axis=1)[:, np.newaxis]
    # Calculate initial vertex coordinate system
    for i in range(FV['vertices'].shape[0]):
        up[i, :] = np.random.rand(3)*2-1
        up[i, :] = np.cross(up[i, :], VertexNormals[i, :])
        up[i, :] /= np.linalg.norm(up[i, :])
        vp[i, :] = np.cross(VertexNormals[i, :], up[i, :])

    print('Finished calculating vertex normals')
    
    return VertexNormals, Avertex, Acorner, up, vp

def project_curvature_tensor(uf, vf, nf, old_ku, old_kuv, old_kv, up, vp):
    """
    Perform a projection of the tensor variables to the vertex coordinate system.
    
    Parameters:
    uf, vf (numpy.ndarray): Face coordinate system unit vectors.
    nf (numpy.ndarray): Normal vector of the face.
    old_ku, old_kuv, old_kv (float): Face curvature tensor variables.
    up, vp (numpy.ndarray): Vertex coordinate system unit vectors.
    
    Returns:
    new_ku, new_kuv, new_kv (float): Vertex curvature tensor variables.
    """
    
    # Rotate the coordinate system
    r_new_u, r_new_v = rotate_coord_system(up, vp, nf)
    
    # Curvature tensor matrix for the face
    old_tensor = np.array([[old_ku, old_kuv], 
                           [old_kuv, old_kv]])
    
    # Project coordinates
    u1 = np.dot(r_new_u, uf)
    v1 = np.dot(r_new_u, vf)
    u2 = np.dot(r_new_v, uf)
    v2 = np.dot(r_new_v, vf)

    # Calculate new curvature tensor variables
    new_ku = np.dot([u1, v1], np.dot(old_tensor, [u1, v1]))
    new_kuv = np.dot([u1, v1], np.dot(old_tensor, [u2, v2]))
    new_kv = np.dot([u2, v2], np.dot(old_tensor, [u2, v2]))

    return new_ku, new_kuv, new_kv

def rotate_coord_system_vectorized(u,v,nf):
    """
    rotate the vectors u and v to the plane of normal nf
    Args:
        u (np.array (N,3)) : 1st set of vector to be rotated
        v (np.array (N,3)) : 2nd set of vector to be rotated
        nf (np.array (N,3)) : normal vectors of the plane to rotate into
    Output:
        new_u, new_v (np.array(N, 3)) : rotated u and v (orthogonal to nf)
    """
    n = np.cross(u,v)
    n = n/np.linalg.norm(n, axis=1)[:,None]

    dot = np.sum(nf*n, axis=1)[:,None] # np.dot(nf,n) : nf.n
    perp = nf - dot*n
    dperp = (nf + n)/(1+dot)
    new_u = u - dperp * np.sum(perp*u, axis=1)[:,None] # u - dperp * (perp.u)
    new_v = v - dperp * np.sum(perp*v, axis=1)[:,None]

    threshold = np.where(dot<=-1)[0]
    new_u[threshold] = -u[threshold]
    new_v[threshold] = -v[threshold]

    return new_u, new_v


def rotate_coord_system(u,v,nf):
    """
    rotate the vectors u and v to the plane of normal nf
    Args:
        u (np.array (N,3)) : 1st set of vector to be rotated
        v (np.array (N,3)) : 2nd set of vector to be rotated
        nf (np.array (N,3)) : normal vectors of the plane to rotate into
    Output:
        new_u, new_v (np.array(N, 3)) : rotated u and v (orthogonal to nf)
    """
    n = np.cross(u,v)
    n = n/np.linalg.norm(n)

    dot = np.dot(nf,n)
    if dot <= -1:
        return -u,-v
    
    perp = nf - dot*n
    dperp = (nf + n)/(1+dot)
    new_u = u - dperp * np.dot(perp,u) # u - dperp * (perp.u)
    new_v = v - dperp * np.dot(perp,v)

    return new_u, new_v



def calc_curvature(FV, VertexNormals, FaceNormals, Avertex, Acorner, up, vp):
    """
    Calculate the second fundamental matrix and curvature using least squares.
    
    Parameters:
    FV (dict): Face-vertex data structure containing a list of vertices and a list of faces.
    VertexNormals (numpy.ndarray): n x 3 matrix (n = number of vertices) containing the normal at each vertex.
    FaceNormals (numpy.ndarray): m x 3 matrix (m = number of faces) containing the normal of each face.
    Avertex (numpy.ndarray): Voronoi area at each vertex.
    Acorner (numpy.ndarray): Voronoi area slice at each face corner.
    up (numpy.ndarray): Vertex coordinate system (u vector).
    vp (numpy.ndarray): Vertex coordinate system (v vector).
    
    Returns:
    FaceSFM (list): List of m elements containing the second fundamental matrix for each face.
    VertexSFM (list): List of n elements containing the second fundamental matrix for each vertex.
    wfp (numpy.ndarray): Corner Voronoi weights.
    """
    print('Calculating Curvature Tensors... Please wait')
    
    # Initialize the second fundamental matrix for faces and vertices
    FaceSFM = [np.zeros((2, 2)) for _ in range(FV['faces'].shape[0])]
    VertexSFM = [np.zeros((2, 2)) for _ in range(FV['vertices'].shape[0])]

    # Get all edge vectors
    e0 = FV['vertices'][FV['faces'][:, 2], :] - FV['vertices'][FV['faces'][:, 1], :]
    e1 = FV['vertices'][FV['faces'][:, 0], :] - FV['vertices'][FV['faces'][:, 2], :]
    e2 = FV['vertices'][FV['faces'][:, 1], :] - FV['vertices'][FV['faces'][:, 0], :]

    # Normalize edge vectors
    e0_norm = e0 / np.linalg.norm(e0, axis=1)[:, np.newaxis]
    # e1_norm = e1 / np.linalg.norm(e1, axis=1)[:, np.newaxis]
    # e2_norm = e2 / np.linalg.norm(e2, axis=1)[:, np.newaxis]

    wfp = np.zeros((FV['faces'].shape[0], 3))
    # Iterate over each face
    for i in range(FV['faces'].shape[0]):
        # Calculate Curvature Per Face
        nf = FaceNormals[i, :]
        t = e0_norm[i, :].T
        B = np.cross(nf, t).T
        B = B / np.linalg.norm(B)

        # Extract relevant normals in face vertices
        n0 = VertexNormals[FV['faces'][i, 0], :]
        n1 = VertexNormals[FV['faces'][i, 1], :]
        n2 = VertexNormals[FV['faces'][i, 2], :]

        # Solve least squares problem of the form Ax = b
        # A = np.array([
        #     [np.dot(e0[i, :], t), np.dot(e0[i, :], B), 0],
        #     [0, np.dot(e0[i, :], t), np.dot(e0[i, :], B)],
        #     [np.dot(e1[i, :], t), np.dot(e1[i, :], B), 0],
        #     [0, np.dot(e1[i, :], t), np.dot(e1[i, :], B)],
        #     [np.dot(e2[i, :], t), np.dot(e2[i, :], B), 0],
        #     [0, np.dot(e2[i, :], t), np.dot(e2[i, :], B)]
        # ])
        
        b = np.array([
            np.dot(n2 - n1, t),
            np.dot(n2 - n1, B),
            np.dot(n0 - n2, t),
            np.dot(n0 - n2, B),
            np.dot(n1 - n0, t),
            np.dot(n1 - n0, B)
        ])

        # Solve the system A * x = b
        # x,_,_,_ = np.linalg.lstsq(A, b, rcond=None)

        x = custom_solve_Ab(a=np.dot(e0[i, :], t), b=np.dot(e0[i, :], B), c=np.dot(e1[i, :], t), d=np.dot(e1[i, :], B), e=np.dot(e2[i, :], t), f=np.dot(e2[i, :], B), vec_b=b)

        # Construct the face second fundamental matrix
        FaceSFM[i] = np.array([[x[0], x[1]], [x[1], x[2]]])
        
        # Calculate Curvature Per Vertex
        wfp[i, 0] = Acorner[i, 0] / Avertex[FV['faces'][i, 0]]
        wfp[i, 1] = Acorner[i, 1] / Avertex[FV['faces'][i, 1]]
        wfp[i, 2] = Acorner[i, 2] / Avertex[FV['faces'][i, 2]]
        
        # Calculate new coordinate system and project the tensor
        for j in range(3):
            new_ku, new_kuv, new_kv = project_curvature_tensor(
                t, B, nf, x[0], x[1], x[2], 
                up[FV['faces'][i, j], :], 
                vp[FV['faces'][i, j], :]
            )
            VertexSFM[FV['faces'][i, j]] += wfp[i, j] * np.array([[new_ku, new_kuv], [new_kuv, new_kv]])

    print('Finished Calculating Curvature Tensors.')
    return FaceSFM, VertexSFM, wfp


def custom_solve_Ab(a,b,c,d,e,f, vec_b):
    aa, bb, cc, dd, ee, ff = a*a, b*b, c*c, d*d, e*e, f*f
    aaa, bbb, ccc, ddd, eee, fff = aa*a, bb*b, cc*c, dd*d, ee*e, ff*f
    aaaa, bbbb, cccc, dddd, eeee, ffff = aaa*a, bbb*b, ccc*c, ddd*d, eee*e, fff*f

    denom = aaaa*dd + aaaa*ff - 2*aaa*b*c*d - 2*aaa*b*e*f + aa*bb*cc + aa*bb*dd + aa*bb*ee + aa*bb*ff + aa*cc*dd + 2*aa*cc*ff - 2*aa*c*d*e*f + aa*dddd + 2*aa*dd*ee + 2*aa*dd*ff + aa*ee*ff + aa*ffff - 2*a*bbb*c*d - 2*a*bbb*e*f - 2*a*b*ccc*d - 2*a*b*cc*e*f - 2*a*b*c*ddd - 2*a*b*c*d*ee - 2*a*b*c*d*ff - 2*a*b*dd*e*f - 2*a*b*eee*f - 2*a*b*e*fff + bbbb*cc + bbbb*ee + bb*cccc + bb*cc*dd + 2*bb*cc*ee + 2*bb*cc*ff - 2*bb*c*d*e*f + 2*bb*dd*ee + bb*eeee + bb*ee*ff + cccc*ff - 2*ccc*d*e*f + cc*dd*ee + cc*dd*ff + cc*ee*ff + cc*ffff - 2*c*ddd*e*f - 2*c*d*eee*f - 2*c*d*e*fff + dddd*ee + dd*eeee + dd*ee*ff
    m00 = aa*dd + aa*ff - 2*a*b*c*d - 2*a*b*e*f + bbbb + bb*cc + 2*bb*dd + bb*ee + 2*bb*ff + cc*ff - 2*c*d*e*f + dddd + dd*ee + 2*dd*ff + ffff
    m01 = -a*bbb - a*b*dd - a*b*ff - bb*c*d - bb*e*f - c*ddd - c*d*ff - dd*e*f - e*fff
    m02 = aa*bb + 2*a*b*c*d + 2*a*b*e*f + cc*dd + 2*c*d*e*f + ee*ff
    # m10 = m01
    m11 = aa*bb + aa*dd + aa*ff + bb*cc + bb*ee + cc*dd + cc*ff + dd*ee + ee*ff
    m12 = -aaa*b - aa*c*d - aa*e*f - a*b*cc - a*b*ee - ccc*d - cc*e*f - c*d*ee - eee*f
    # m20 = m02
    # m21 = m12
    m22 = aaaa + 2*aa*cc + aa*dd + 2*aa*ee + aa*ff - 2*a*b*c*d - 2*a*b*e*f + bb*cc + bb*ee + cccc + 2*cc*ee + cc*ff - 2*c*d*e*f + dd*ee + eeee

    Mt = np.array([
        [a,0,c,0,e,0],
        [b,a,d,c,f,e],
        [0,b,0,d,0,f]
        ])
    MtM_inv = (1/denom) * np.array([
        [m00, m01, m02],
        [m01, m11, m12],
        [m02, m12, m22]
    ])

    return MtM_inv @ Mt @ vec_b


def get_principal_curvatures(FV, VertexSFM, up, vp):
    """
    Calculate the principal curvatures and their directions from the second fundamental matrix.
    
    Parameters:
    FV (dict): Mesh face-vertex data structure.
    VertexSFM (list): Second fundamental matrix for each vertex.
    up (numpy.ndarray): Vertex local coordinate frame (u vector).
    vp (numpy.ndarray): Vertex local coordinate frame (v vector).
    
    Returns:
    PrincipalCurvatures (numpy.ndarray): 2 x [number of vertices] matrix containing the principal curvature values.
    PrincipalDir1 (numpy.ndarray): Direction vectors of the first principal component.
    PrincipalDir2 (numpy.ndarray): Direction vectors of the second principal component.
    """
    print('Calculating Principal Components')

    # Initialize arrays for principal curvatures and directions
    PrincipalCurvatures = np.zeros((2, FV['vertices'].shape[0]))
    PrincipalDir1 = np.zeros((FV['vertices'].shape[0], 3))
    PrincipalDir2 = np.zeros((FV['vertices'].shape[0], 3))

    # Iterate over each vertex
    for i in range(FV['vertices'].shape[0]):
        # Compute the normal vector np
        np_vector = np.cross(up[i, :], vp[i, :])

        # Rotate the coordinate system to align with the normal
        r_old_u, r_old_v = rotate_coord_system(up[i, :], vp[i, :], np_vector)

        # Retrieve components of the second fundamental matrix
        ku = VertexSFM[i][0, 0]
        kuv = VertexSFM[i][0, 1]
        kv = VertexSFM[i][1, 1]

        # Initialize rotation parameters
        c = 1
        s = 0
        tt = 0

        if kuv != 0:
            # Jacobi rotation to diagonalize the second fundamental matrix
            h = 0.5 * (kv - ku) / kuv
            if h < 0:
                tt = 1 / (h - np.sqrt(1 + h * h))
            else:
                tt = 1 / (h + np.sqrt(1 + h * h))

            # Calculate cosine and sine for the rotation
            c = 1 / np.sqrt(1 + tt * tt)
            s = tt * c

        # Compute the principal curvatures
        k1 = ku - tt * kuv
        k2 = kv + tt * kuv

        # Determine the direction of the principal components
        if abs(k1) >= abs(k2):
            PrincipalDir1[i, :] = c * r_old_u - s * r_old_v
        else:
            # Swap the principal curvatures if necessary
            k1, k2 = k2, k1
            PrincipalDir1[i, :] = s * r_old_u + c * r_old_v

        # Compute the second principal direction
        PrincipalDir2[i, :] = np.cross(np_vector, PrincipalDir1[i, :])

        # Store the principal curvature values
        PrincipalCurvatures[0, i] = k1
        PrincipalCurvatures[1, i] = k2

        # Check for NaN values
        if np.isnan(k1) or np.isnan(k2):
            print('NAN')

    print('Finished Calculating Principal Components')
    return PrincipalCurvatures, PrincipalDir1, PrincipalDir2


def project_c_tensor(uf, vf, nf, Old_C, up, vp):
    """
    Project the curvature tensor variables from the face coordinate system to the vertex coordinate system.
    
    Parameters:
    uf (numpy.ndarray): Face coordinate system (u vector).
    vf (numpy.ndarray): Face coordinate system (v vector).
    nf (numpy.ndarray): Face normal vector.
    Old_C (numpy.ndarray): Face curvature tensor variables as a vector.
    up (numpy.ndarray): Vertex coordinate system (u vector).
    vp (numpy.ndarray): Vertex coordinate system (v vector).
    
    Returns:
    new_C (numpy.ndarray): Vertex curvature tensor variables as a column vector.
    """
    
    # Initialize a tensor matrix (not used but keeping the placeholder for potential future use)
    new_CMatrix = np.zeros((2, 2, 2))

    # Rotate the coordinate system to align with the face normal
    r_new_u, r_new_v = rotate_coord_system(up, vp, nf)

    # Compute the projection coefficients
    u1 = np.dot(r_new_u, uf)
    v1 = np.dot(r_new_u, vf)
    u2 = np.dot(r_new_v, uf)
    v2 = np.dot(r_new_v, vf)

    # Calculate the new curvature tensor components
    new_C = np.zeros(4)
    new_C[0] = Old_C[0] * u1 * u1 * u1 + 3 * Old_C[1] * u1 * u1 * v1 + 3 * Old_C[2] * u1 * v1 * v1 + Old_C[3] * v1 * v1 * v1
    new_C[1] = (Old_C[0] * u2 * u1 * u1 +
                Old_C[1] * (v2 * u1 * u1 + 2 * u2 * u1 * v1) +
                Old_C[2] * (u2 * v1 * v1 + 2 * u1 * v1 * v2) +
                Old_C[3] * v2 * v1 * v1)
    new_C[2] = (Old_C[0] * u1 * u2 * u2 +
                Old_C[1] * (v1 * u2 * u2 + 2 * u2 * u1 * v2) +
                Old_C[2] * (u1 * v2 * v2 + 2 * u2 * v2 * v1) +
                Old_C[3] * v1 * v2 * v2)
    new_C[3] = Old_C[0] * u2 * u2 * u2 + 3 * Old_C[1] * u2 * u2 * v2 + 3 * Old_C[2] * u2 * v2 * v2 + Old_C[3] * v2 * v2 * v2

    return new_C


import numpy as np

def calc_curvature_derivative(FV, FaceNormals, PrincipalCurvatures, up, vp, wfp):
    """
    Calculate the curvature derivative matrix (2x2x2) for each face and vertex in the mesh.
    
    Parameters:
    FV (dict): Face-vertex data structure containing a list of vertices and faces.
    FaceNormals (numpy.ndarray): Array containing the normals of each face.
    PrincipalCurvatures (numpy.ndarray): 2x[number of vertices] matrix with principal curvature values.
    up, vp (numpy.ndarray): Arrays defining the vertex local coordinate frames.
    wfp (numpy.ndarray): Corner Voronoi weights.
    
    Returns:
    FaceCMatrix (numpy.ndarray): m x (2x2x2) array with curvature derivative tensor for each face.
    VertexCMatrix (numpy.ndarray): n x (2x2x2) array with curvature derivative tensor for each vertex.
    Cmagnitude (numpy.ndarray): Curvature magnitude for each vertex.
    """

    print('Calculating C Tensors... Please wait')

    # Initialization
    FaceCMatrix = np.zeros((FV['faces'].shape[0], 4))
    VertexCMatrix = np.zeros((FV['vertices'].shape[0], 4))
    FV_SFM = [np.zeros((2, 2)) for _ in range(3)]
    # Cmagnitude = np.zeros(FV['vertices'].shape[0])

    # Get all edge vectors
    e0 = FV['vertices'][FV['faces'][:, 2], :] - FV['vertices'][FV['faces'][:, 1], :]
    e1 = FV['vertices'][FV['faces'][:, 0], :] - FV['vertices'][FV['faces'][:, 2], :]
    e2 = FV['vertices'][FV['faces'][:, 1], :] - FV['vertices'][FV['faces'][:, 0], :]

    # Normalize edge vectors
    e0_norm = e0 / np.linalg.norm(e0, axis=1)[:, np.newaxis]
    # e1_norm = e1 / np.linalg.norm(e1, axis=1)[:, np.newaxis]
    # e2_norm = e2 / np.linalg.norm(e2, axis=1)[:, np.newaxis]

    for i in range(FV['faces'].shape[0]):
        # Set face coordinate frame
        nf = FaceNormals[i, :]
        t = e0_norm[i, :].T
        B = np.cross(nf, t)
        B = B / np.linalg.norm(B)

        # Solve least squares problem Ax = b
        u = np.array([np.dot(e0[i, :], t), np.dot(e1[i, :], t), np.dot(e2[i, :], t)])
        v = np.array([np.dot(e0[i, :], B), np.dot(e1[i, :], B), np.dot(e2[i, :], B)])

        for j in range(3):
            np_vec = np.cross(up[FV['faces'][i, j], :], vp[FV['faces'][i, j], :])
            np_vec = np_vec / np.linalg.norm(np_vec)

            k1 = PrincipalCurvatures[0, FV['faces'][i, j]]
            k2 = PrincipalCurvatures[1, FV['faces'][i, j]]

            new_ku, new_kuv, new_kv = project_curvature_tensor(
                up[FV['faces'][i, j], :].T, vp[FV['faces'][i, j], :].T, np_vec, k1, 0, k2, t.T, B.T
            )
            FV_SFM[j] = np.array([[new_ku, new_kuv], [new_kuv, new_kv]])

        # Compute changes in curvature
        Delta_e = np.array([
            FV_SFM[2][0, 0] - FV_SFM[1][0, 0],
            FV_SFM[0][0, 0] - FV_SFM[2][0, 0],
            FV_SFM[1][0, 0] - FV_SFM[0][0, 0]
        ])
        Delta_f = np.array([
            FV_SFM[2][0, 1] - FV_SFM[1][0, 1],
            FV_SFM[0][0, 1] - FV_SFM[2][0, 1],
            FV_SFM[1][0, 1] - FV_SFM[0][0, 1]
        ])
        Delta_g = np.array([
            FV_SFM[2][1, 1] - FV_SFM[1][1, 1],
            FV_SFM[0][1, 1] - FV_SFM[2][1, 1],
            FV_SFM[1][1, 1] - FV_SFM[0][1, 1]
        ])

        # Least squares matrix setup
        sumU2 = np.dot(u, u)
        sumV2 = np.dot(v, v)
        sumUV = np.dot(u, v)
        A = np.array([
            [sumU2, sumUV, 0, 0],
            [sumUV, 2 * sumU2 + sumV2, 2 * sumUV, 0],
            [0, 2 * sumUV, sumU2 + 2 * sumV2, sumUV],
            [0, 0, sumUV, sumV2]
        ])
        b = np.array([
            np.dot(Delta_e, u), 
            np.dot(Delta_e, v) + 2 * np.dot(Delta_f, u), 
            2 * np.dot(Delta_f, v) + np.dot(Delta_g, u), 
            np.dot(Delta_g, v)
        ])
        x,_,_,_ = np.linalg.lstsq(A, b, rcond=None)
        # x = np.linalg.solve(A, b)
        FaceCMatrix[i, :] = x

        # Project the tensor for each vertex and update the VertexCMatrix
        for j in range(3):
            new_CMatrix = project_c_tensor(t, B, nf, x, up[FV['faces'][i, j], :], vp[FV['faces'][i, j], :])
            VertexCMatrix[FV['faces'][i, j], :] += wfp[i, j] * new_CMatrix

    # Calculate the magnitude of the curvature derivative
    a = VertexCMatrix[:, 0]
    b = VertexCMatrix[:, 1]
    c = VertexCMatrix[:, 2]
    d = VertexCMatrix[:, 3]


    Cmagnitude = a ** 2 + 3 * b ** 2 + 3 * c ** 2 + d ** 2
    epsilon = 1e-5
    Cmagnitude[Cmagnitude < epsilon] = 0

    print('Finished Calculating C Tensors.')

    return FaceCMatrix, VertexCMatrix, Cmagnitude


FV = {'vertices':np.array([v.co for v in bm.verts]), "faces":np.array([[v.index for v in f.verts] for f in bm.faces])}

face_n = get_face_normals(bm)
VertexNormals, Avertex, Acorner, up, vp = calc_vertex_normals(FV, face_n)
VertexNormals = np.array([v.normal for v in bm.verts])
Avertex = bm.vertex_attributes["vertex_area"]
Acorner = np.array([[bm.fc_areas[fc] for fc in f.loops] for f in bm.faces])
FaceSFM, VertexSFM, wfp = calc_curvature(FV, VertexNormals, face_n, Avertex, Acorner, up, vp)
PrincipalCurvatures, PrincipalDir1, PrincipalDir2 = get_principal_curvatures(FV, VertexSFM, up, vp)
# FaceCMatrix, VertexCMatrix, Cmagnitude = calc_curvature_derivative(FV, face_n, PrincipalCurvatures, up, vp, wfp)


Calculating vertex normals... Please wait
Finished calculating vertex normals
Calculating Curvature Tensors... Please wait
Finished Calculating Curvature Tensors.
Calculating Principal Components
Finished Calculating Principal Components


In [51]:
from collections import deque
my_queue = deque()
my_queue.append(bm.verts[0])
counter = 0
change = 0
already_visited = np.zeros(len(bm.verts), dtype=bool)
while len(my_queue) > 0 and counter < 1000000:
    v = my_queue.popleft()
    already_visited[v.index] = True
    v_e2 =PrincipalDir2[v.index]
    for e in v.link_edges:
        ov = e.other_vert(v)
        if ov.index < v.index:
            continue
        ov_e2 =PrincipalDir2[ov.index]
        if np.dot(v_e2, ov_e2) < 0:
            change+=1
            PrincipalDir2[ov.index] = - ov_e2
        my_queue.append(ov)
        counter+=1
    if len(my_queue)==0:
        other_island_index = np.where(already_visited==0)[0]
        if len(other_island_index)>0:
            my_queue.append(bm.verts[other_island_index[0]])
print(change, counter)

mesh.attributes.remove(mesh.attributes["e2"])
mesh.attributes.remove(mesh.attributes["e1"])

attr = mesh.attributes.new(name="e1", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector',PrincipalDir1.flatten())

attr = mesh.attributes.new(name="e2", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector',PrincipalDir2.flatten())
        

# attr = mesh.attributes.new(name="true_e1", type='FLOAT_VECTOR', domain='POINT')
# attr.data.foreach_set('vector',PrincipalDir1.flatten())

# attr = mesh.attributes.new(name="true_e2", type='FLOAT_VECTOR', domain='POINT')
# attr.data.foreach_set('vector',PrincipalDir2.flatten())
        

27258 263031


In [13]:
attr = mesh.attributes.new(name="VertexNormal", type='FLOAT_VECTOR', domain='POINT')
attr.data.foreach_set('vector',VertexNormals.flatten())

In [ ]:
attr = mesh.attributes.new(name="k1", type='FLOAT', domain='POINT')
attr.data.foreach_set('value',PrincipalCurvatures[0])

attr = mesh.attributes.new(name="k2", type='FLOAT', domain='POINT')
attr.data.foreach_set('value',PrincipalCurvatures[1])

In [144]:
VertexNormals

array([[ 0.59448085, -0.57389727, -0.56323569],
       [-0.59448134, -0.57389608, -0.56323638],
       [ 0.60127008, -0.60064682, -0.52697029],
       ...,
       [ 0.1281804 ,  0.47404716, -0.87111944],
       [ 0.13147319,  0.34648071, -0.92879811],
       [ 0.14091302,  0.42985916, -0.89183217]])